In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/glove840b300dtxt/glove.840B.300d.txt
/kaggle/input/jigsaw-multilingual-toxic-test-translated/jigsaw_miltilingual_test_translated.csv
/kaggle/

In [3]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing,model_selection, metrics
from keras.models import Sequential
from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from tqdm import tqdm

print("Imported")

Imported


In [4]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [6]:
train = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
val = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
print("Done")

Done


In [7]:
train = train.loc[:10000,:]
train.shape

(10001, 8)

In [8]:
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
9996,1a7a4868968e2b9e,"Those two love to disagree, don't they? 206.17...",0,0,0,0,0,0
9997,1a7c3bec9a71415d,"""I have changed """"Lance Thomas"""" to """"Lance Th...",0,0,0,0,0,0
9998,1a7c9c14b0cf0fe0,states \n\nCourts: I have been putting all art...,0,0,0,0,0,0
9999,1a7d550fec6e9777,Will do buddy ). But what is this thing about ...,0,0,0,0,0,0
10000,1a7d7c88372e5668,Hi RedRose and apologies for delay. Here is a ...,0,0,0,0,0,0


In [9]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
train_X, valid_X, train_y, valid_y = train_test_split(train.comment_text.values, train.toxic.values, stratify=train.toxic.values, random_state=123, 
                                                      train_size=0.8, shuffle=True)
train_X.shape

(8000,)

In [11]:

lenmax = train['comment_text'].apply(lambda x: len(str(x).split())).max()
lenmax

1403

In [14]:

tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(list(train_X) + list(valid_X))

seq_trainx = tokenizer.texts_to_sequences(train_X)
seq_validx = tokenizer.texts_to_sequences(valid_X)

#zero p
pad_trainx = sequence.pad_sequences(seq_trainx, maxlen=lenmax)
pad_validx = sequence.pad_sequences(seq_validx, maxlen=lenmax)

word_index = tokenizer.word_index

[28,
 135,
 7,
 567,
 11,
 25647,
 35,
 8,
 20,
 13,
 1,
 4161,
 3,
 1,
 6003,
 3895,
 16,
 600,
 2,
 1626,
 1074,
 12,
 15,
 5,
 586,
 3509,
 409,
 41,
 199,
 274]

In [ ]:
embeddings_index = {}
f = open('/kaggle/input/glove840b300dtxt/glove.840B.300d.txt','r')
for line in tqdm(f):
    words = line.split(' ')
    word = words[0]
    c = np.asarray([float(val) for val in words[1:]])
    embeddings_index[word] = c
f.close()

In [ ]:
embedding_mat = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vec = embeddings_index.get(word)
    if embedding_vec is not None:
        embedding_mat[i] = embedding_vec

In [ ]:
with strategy.scope():
    
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_mat],
                     input_length=lenmax,
                     trainable=False))

    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(pad_trainx,train_y,nb_epoch=5, batch_size=64*strategy.num_replicas_in_sync)


In [ ]:
predict = model.predict(pad_validx)
fpr, tpr, thresholds = metrics.roc_curve(valid_y, predict)
roc_auc = metrics.auc(fpr, tpr)
roc_auc